In [1]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
#import kalshi_python

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2ApiKey import ExchangeClient
import time
import json
import uuid

In [2]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key

In [3]:
import os

prod_key_id = os.getenv('KALSHI_API_KEY_ID')  # read from environment variable
prod_private_key = load_private_key_from_file('../../kalshi-key.key')

# Updated API base URLs
kalshi_api_base = "https://api.elections.kalshi.com/trade-api/v2"  # New Kalshi markets
legacy_api_base = "https://trading-api.kalshi.com/trade-api/v2"    # Legacy markets

# Initialize client with new Kalshi markets URL for production
exchange_client = ExchangeClient(
    exchange_api_base=kalshi_api_base,
    key_id=prod_key_id,
    private_key=prod_private_key
)

# Check connection and balance
print("Exchange Status:", exchange_client.get_exchange_status())
print("\nBalance:", exchange_client.get_balance())

Exchange Status: {'exchange_active': True, 'trading_active': True}

Balance: {'balance': 1744}


In [4]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {'limit':100,
                    'cursor':None, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker':None,
                    'max_close_ts':None, # pass in unix_ts
                    'min_close_ts':None, # pass in unix_ts
                    'status':None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in payload:', markets_response['markets'][0])
print()
print('cursor:', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in payload: {'ticker': 'KXHIGHMIA-24NOV07-T87', 'event_ticker': 'KXHIGHMIA-24NOV07', 'market_type': 'binary', 'title': 'Will the **high temp in Miami** be >87° on Nov 7, 2024?', 'subtitle': '88° or above', 'yes_sub_title': '88° or above', 'no_sub_title': '88° or above', 'open_time': '2024-11-06T15:00:00Z', 'close_time': '2024-11-08T04:59:00Z', 'expected_expiration_time': '2024-11-08T15:00:00Z', 'expiration_time': '2024-11-14T15:00:00Z', 'latest_expiration_time': '2024-11-14T15:00:00Z', 'settlement_timer_seconds': 3600, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 1, 'yes_ask': 71, 'no_bid': 29, 'no_ask': 99, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 2347960, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': 'Climate and We

In [5]:
import json
from pprint import pprint
import time

# Get list of orders for PRES-2024-DJT
try:
    # Filter orders specifically for PRES-2024-DJT
    order_params = {
        'limit': 100,
        'cursor': None,
        'ticker': 'PRES-2024-DJT',  # Specifically filter for this market
        'event_ticker': None,
        'min_ts': None,
        'max_ts': None
    }
    
    orders_response = exchange_client.get_orders(**order_params)
    
    print("Orders Response for PRES-2024-DJT:")
    print(f"Number of orders: {len(orders_response['orders'])}")
    print("\nResponse keys:", orders_response.keys())
    
    # Print first order details if any exist
    if orders_response['orders']:
        print("\nFirst order details:")
        pprint(orders_response['orders'][0])
    else:
        print("\nNo orders found for PRES-2024-DJT")
        
    # Print cursor for pagination if it exists
    if 'cursor' in orders_response:
        print("\nNext page cursor:", orders_response['cursor'])
        
except Exception as e:
    print(f"Error getting orders: {e}")

Orders Response for PRES-2024-DJT:
Number of orders: 4

Response keys: dict_keys(['orders', 'cursor'])

First order details:
{'action': 'buy',
 'amend_count': 0,
 'amend_taker_fill_count': 0,
 'client_order_id': '',
 'close_cancel_count': 0,
 'created_time': '2024-11-06T11:50:02.384553Z',
 'decrease_count': 0,
 'expiration_time': '2024-11-06T06:50:02Z',
 'fcc_cancel_count': 0,
 'last_update_time': '2024-11-06T11:50:02.384553Z',
 'maker_fees': 0,
 'maker_fill_cost': 0,
 'maker_fill_count': 0,
 'maker_self_trade_cancel_count': 0,
 'no_price': 2,
 'order_group_id': '',
 'order_id': 'e751b680-afc8-47a9-92c5-222bdf84d397',
 'place_count': 0,
 'queue_position': 0,
 'remaining_count': 0,
 'self_trade_prevention_type': '',
 'side': 'yes',
 'status': 'executed',
 'taker_fees': 0,
 'taker_fill_cost': 9996,
 'taker_fill_count': 102,
 'taker_self_trade_cancel_count': 0,
 'ticker': 'PRES-2024-DJT',
 'type': 'market',
 'user_id': '988c42a1-07ad-4dcf-b103-f4a07395ba6f',
 'yes_price': 98}

Next page c

In [6]:
# Get candlesticks for PRES-2024-DJT and save to CSV
import pandas as pd
from datetime import datetime
import os

try:
    # Calculate proper time range based on period_interval
    period_interval = 60  # 1 hour intervals
    max_periods = 5000
    
    # Calculate end_ts as current time
    end_ts = int(time.time())
    max_duration = period_interval * 60 * max_periods
    start_ts = end_ts - max_duration
    
    candlesticks = exchange_client.get_candlesticks(
        series_ticker='PRES',
        ticker='PRES-2024-DJT',
        start_ts=start_ts,
        end_ts=end_ts,
        period_interval=period_interval
    )
    
    # Convert candlesticks data to DataFrame
    df = pd.DataFrame(candlesticks['candlesticks'])
    
    # Create new columns for each nested value
    df['timestamp'] = pd.to_datetime(df['end_period_ts'], unit='s')
    
    # Extract bid values
    df['bid_open'] = df['yes_bid'].apply(lambda x: x['open'])
    df['bid_low'] = df['yes_bid'].apply(lambda x: x['low'])
    df['bid_high'] = df['yes_bid'].apply(lambda x: x['high'])
    df['bid_close'] = df['yes_bid'].apply(lambda x: x['close'])
    
    # Extract ask values
    df['ask_open'] = df['yes_ask'].apply(lambda x: x['open'])
    df['ask_low'] = df['yes_ask'].apply(lambda x: x['low'])
    df['ask_high'] = df['yes_ask'].apply(lambda x: x['high'])
    df['ask_close'] = df['yes_ask'].apply(lambda x: x['close'])
    
    # Extract price values
    df['price_open'] = df['price'].apply(lambda x: x['open'])
    df['price_low'] = df['price'].apply(lambda x: x['low'])
    df['price_high'] = df['price'].apply(lambda x: x['high'])
    df['price_close'] = df['price'].apply(lambda x: x['close'])
    df['price_mean'] = df['price'].apply(lambda x: x['mean'])
    
    # Select and reorder columns
    cleaned_df = df[[
        'timestamp',
        'bid_open', 'bid_low', 'bid_high', 'bid_close',
        'ask_open', 'ask_low', 'ask_high', 'ask_close',
        'price_open', 'price_low', 'price_high', 'price_close', 'price_mean',
        'volume', 'open_interest'
    ]]
    
    # Create data directory if it doesn't exist
    data_dir = "../../app/data/kalshi"
    os.makedirs(data_dir, exist_ok=True)
    
    # Save to CSV with timestamp in filename
    filename = os.path.join(data_dir, f"PRES-2024-DJT_candlesticks.csv")
    cleaned_df.to_csv(filename, index=False)
    
    print(f"\nCandlesticks data saved to {filename}")
    print(f"Number of periods saved: {len(cleaned_df)}")
    print("\nFirst candlestick:")
    print(cleaned_df.iloc[0])
    
except Exception as e:
    print(f"Error getting/saving candlesticks: {e}")


Candlesticks data saved to ../../data/kalshi/PRES-2024-DJT_candlesticks_20241106_122025.csv
Number of periods saved: 612

First candlestick:
timestamp        2024-10-04 13:00:00
bid_open                          49
bid_low                           49
bid_high                          49
bid_close                         49
ask_open                         100
ask_low                           50
ask_high                         100
ask_close                         50
price_open                      49.0
price_low                       49.0
price_high                      51.0
price_close                     50.0
price_mean                      50.0
volume                          1715
open_interest                   1659
Name: 0, dtype: object


In [7]:
# Get candlesticks for PRES-2024-KH and save to CSV
import pandas as pd
from datetime import datetime
import os

try:
    # Calculate proper time range based on period_interval
    period_interval = 60  # 1 hour intervals
    max_periods = 5000
    
    # Calculate end_ts as current time
    end_ts = int(time.time())
    max_duration = period_interval * 60 * max_periods
    start_ts = end_ts - max_duration
    
    candlesticks = exchange_client.get_candlesticks(
        series_ticker='PRES',
        ticker='PRES-2024-KH',
        start_ts=start_ts,
        end_ts=end_ts,
        period_interval=period_interval
    )
    
    # Convert candlesticks data to DataFrame
    df = pd.DataFrame(candlesticks['candlesticks'])
    
    # Create new columns for each nested value
    df['timestamp'] = pd.to_datetime(df['end_period_ts'], unit='s')
    
    # Extract bid values
    df['bid_open'] = df['yes_bid'].apply(lambda x: x['open'])
    df['bid_low'] = df['yes_bid'].apply(lambda x: x['low'])
    df['bid_high'] = df['yes_bid'].apply(lambda x: x['high'])
    df['bid_close'] = df['yes_bid'].apply(lambda x: x['close'])
    
    # Extract ask values
    df['ask_open'] = df['yes_ask'].apply(lambda x: x['open'])
    df['ask_low'] = df['yes_ask'].apply(lambda x: x['low'])
    df['ask_high'] = df['yes_ask'].apply(lambda x: x['high'])
    df['ask_close'] = df['yes_ask'].apply(lambda x: x['close'])
    
    # Extract price values
    df['price_open'] = df['price'].apply(lambda x: x['open'])
    df['price_low'] = df['price'].apply(lambda x: x['low'])
    df['price_high'] = df['price'].apply(lambda x: x['high'])
    df['price_close'] = df['price'].apply(lambda x: x['close'])
    df['price_mean'] = df['price'].apply(lambda x: x['mean'])
    
    # Select and reorder columns
    cleaned_df = df[[
        'timestamp',
        'bid_open', 'bid_low', 'bid_high', 'bid_close',
        'ask_open', 'ask_low', 'ask_high', 'ask_close',
        'price_open', 'price_low', 'price_high', 'price_close', 'price_mean',
        'volume', 'open_interest'
    ]]
    
    # Create data directory if it doesn't exist
    data_dir = "../../app/data/kalshi"
    os.makedirs(data_dir, exist_ok=True)
    
    # Save to CSV with timestamp in filename
    filename = os.path.join(data_dir, f"PRES-2024-KH_candlesticks.csv")
    cleaned_df.to_csv(filename, index=False)
    
    print(f"\nCandlesticks data saved to {filename}")
    print(f"Number of periods saved: {len(cleaned_df)}")
    print("\nFirst candlestick:")
    print(cleaned_df.iloc[0])
    
except Exception as e:
    print(f"Error getting/saving candlesticks: {e}")


Candlesticks data saved to ../../app/data/kalshi/PRES-2024-KH_candlesticks.csv
Number of periods saved: 611

First candlestick:
timestamp        2024-10-04 13:00:00
bid_open                           0
bid_low                            0
bid_high                          50
bid_close                         49
ask_open                          51
ask_low                           50
ask_high                          51
ask_close                         50
price_open                      51.0
price_low                       49.0
price_high                      51.0
price_close                     50.0
price_mean                      50.0
volume                          2587
open_interest                   2434
Name: 0, dtype: object
